# torch.autograd

- autograd는 torch에서 인공신경망에서 필요한 미분을 자동으로 해주는 부분이다.
- 인공신경망을 다른 말로 말하자면, 어떤 인풋 함수에 중첩으로 적용된 함수의 집합이다.
- 이 함수는 함수의 파라미터(즉 가중치와 편향)로 정의된다.
- 파라미터는 텐서 데이터로 저장된다.
- 인공신경망을 훈련시킬 때는 두 개의 단계를 거친다:
  - Forward Propagation :
    - 인풋 데이터를 함수에 입력하여 출력을 예측한다. 
  - Backward Propagation : 
    - 출력의 오류를 확인하고, 그 오류를 줄이기 위해 실행된 순서의 역방향으로 함수들의 파라미터를 조정한다.
    - 함수의 파라미터의 변화에 대한 오류의 미분을 계산하는데, 이를 그래디언트(gradient)라 부른다.
    - 그리고 이 그래디언트를 사용해 파라미터를 조정하는 것을 그래디언트 하강(gradient descent)라 부른다.
- 인공신경망을 훈련시킬 때는, 이 과정을 여러번 반복하며, 1회의 반복을 1 에포크라 부른다.
- 1 에포크 과정은 pytorch에서 대략적으로 다음과 같은 코드로 나타난다.

In [2]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)
prediction = model(data)
loss = (prediction - labels).sum()
# ----
print(" grad : ", next(model.parameters()).grad)
loss.backward() # Backward Propagation. 오류를 측정하고 모델의 네트워크에 역전파한다. 이 때 Autograd가 미분을 계산한다.
print(" grad : ", next(model.parameters()).grad.shape)
# ----
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
optim.step()

 grad :  None
 grad :  torch.Size([64, 3, 7, 7])


### autograd가 그래디언트를 수집하는 방식

- 일단 파라미터를 나타내는 텐서는 `reuiqres_grad=True`로 설정하여 생성한다.
- autograd가 활성화된 텐서는 어떤 연산이 가해질 때 마다 그 연산의 역사가 기록되어, 나중에 미분에 활용된다.
- $a=(2,3)$, $b=(6,4)$가 인공신경망의 파라미터라고 가정해보자

In [3]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

- 로스 함수 $Q$는 다음과 같이 $a$, $b$에 대한 방정식으로 정의된다고 해보자.
$$Q = 3a^3 - b^2$$
- 이를 계산하면 오류 텐서를 구할 수 있다.
$$
\begin{aligned}
Q = 3a^3 - b^2 &= 3\begin{bmatrix}2^3\\3^3\end{bmatrix} - \begin{bmatrix}6^2\\4^2\end{bmatrix}\\
&= 3\begin{bmatrix}8\\27\end{bmatrix} - \begin{bmatrix}36\\16\end{bmatrix}\\
&= \begin{bmatrix}24\\81\end{bmatrix} - \begin{bmatrix}36\\16\end{bmatrix}\\
&= \begin{bmatrix}-12\\65\end{bmatrix}
\end{aligned}
$$

In [4]:
Q = 3*a**3 - b**2
print(Q)

tensor([-12.,  65.], grad_fn=<SubBackward0>)


- $a$, $b$가 인공신경망의 파라미터이고, $Q$는 오류이다.
- `Q = 3*a**3 - b**2`로 인해 $Q$의 값이 계산할 뿐만 아니라, `Q` 객체에 스스로의 값을 계산하는 대에 쓰인 $a$, $b$ 텐서와 $a$, $b$ 텐서에 적용된 연산(${}^3, {}^2, 3*$, -)을 그래프 자료구조로 기억한다.
- 파라미터에 대한 오류의 변화도를 구하기 위해, $Q$를 $a$, $b$에 대해 각각 편미분 해야한다.

$$
\frac{\partial{Q}}{\partial{a}} = 9a^2\\[1.5em]
\frac{\partial{Q}}{\partial{b}} = -2b
$$

- `Q.backward()`를 호출하면, torch는 $a$, $b$에 대한 $Q$의 편미분을 자동으로 계산하여 `a.grad`, `b.grad`에 각각 저장한다.
- 이 그래디언트들은 $Q$와 동일한 형태(shape)의 텐서이다.
- 이번엔 `Q.backward()`를 호출할 때 `gradient` 인수를 명시적으로 써주어야 한다. 여기선 예시를 위해 $(0.5,1)$을 주었다.

In [5]:
external_grad = torch.tensor([0.5, 1.])
Q.backward(gradient=external_grad) # a.grad, b.grad를 오류의 편미분으로 설정

$$
\begin{array}{c|c}
\begin{aligned}
\left. \frac{\partial{Q}}{\partial{a}} \right|_{a=(2,3)} &= 9\begin{bmatrix}2^2\\3^2\end{bmatrix}\\[1.5em]
&= 9\begin{bmatrix}4\\9\end{bmatrix}\\[1.5em]
&= \begin{bmatrix}36\\81\end{bmatrix}
\end{aligned}&
\begin{aligned}
\left. \frac{\partial{Q}}{\partial{b}} \right|_{b=(6,4)} &= -2\begin{bmatrix}6\\4\end{bmatrix}\\[1.5em]
&= \begin{bmatrix}-12\\-8\end{bmatrix}
\end{aligned}
\end{array}\\[2em]
\nabla Q=\begin{bmatrix}0.5\\1\end{bmatrix}\\[1.5em]
a.grad = \frac{\partial{Q}}{\partial{a}} * \nabla Q = \begin{bmatrix}36\\81\end{bmatrix} * \begin{bmatrix}0.5\\1\end{bmatrix} = \begin{bmatrix}18\\81\end{bmatrix}\\[1.5em]
b.grad = \frac{\partial{Q}}{\partial{b}} * \nabla Q = \begin{bmatrix}-12\\-8\end{bmatrix} * \begin{bmatrix}0.5\\1\end{bmatrix} = \begin{bmatrix}-6\\-8\end{bmatrix}\\[1.5em]
$$


In [6]:
print(9*a**2 == a.grad)
print(a.grad)
print(-2*b == b.grad)
print(b.grad)

tensor([False,  True])
tensor([18., 81.])
tensor([False,  True])
tensor([-6., -8.])


### 연산 그래프(Computational Graph)

- autograd는 유향 비순환 그래프(DAG - Directed Acyclic Graph) 구조로 특정 텐서를 얻어낼 때 까지 수행된 모든 텐서와 연산을 기억하고 있다.
- 이 그래프의 구성요소를 함수 객체(Function objects)라 한다.
  - 예를 들어 "SubBackward", "MulBackward", "PowBackward" 등이 있다
- 이 그래프의 리프는 입력 텐서($a$, $b$)이고, 루트는 출력 텐서($Q$)이다.
- 루트에서 리프까지 역으로 추적하면서 체인 룰(chain rule)을 통해 그래디언트를 계산할 수 있다.
- forward pass에서 autograd는 두 가지 일을 수행한다:
  - 연산을 수행하여 텐서를 계산한다.
  - 연산의 그래디언트 함수(gradient function)를 그래프에 기록한다.
- 어떤 텐서에서 `.backward()`를 호출하면 autograd는 세 가지 일을 수행한다:
  - 각각의 `.grad_fn`에서 그래디언트를 계산한다.
  - 연관된 텐서의 `.grad` 속성에 그 결과를 축적한다.
  - chain rule에 따라 리프 텐서에 이를 때 까지 추적한다.

```mermaid
flowchart BT
 Q --".grad_fn"--> id4["SubBackward()"]
 id4["SubBackward()"] --".next_functions[0][0]"--> id3["MulBackward()"] --".next_functions[0][0]"--> id1["PowBackward()"] --".next_functions[0][0]"--> id6["AccumulateGrad"] --".variable"--> a
 id4["SubBackward()"] --".next_functions[1][0]"--> id2["PowBackward()"] --".next_functions[0][0]"--> id5["AccumulateGrad"] --".variable"-->b
```

- 이 그래프는 루트 텐서 $Q$의 입장에서 바라본 DAG이다.



In [7]:
# Q의 왼쪽 그래프를 따라갑니다.
print(Q) # 3*a^3 - b^2
print(Q.grad_fn) # -
print(Q.grad_fn.next_functions[0][0]) # 3* 
print(Q.grad_fn.next_functions[0][0].next_functions[0][0]) # ^3
print(Q.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0]) # a 래퍼
print(Q.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].variable) # a
print(Q.grad_fn.next_functions[0][0].next_functions[0][0].next_functions[0][0].variable is a) # True

tensor([-12.,  65.], grad_fn=<SubBackward0>)
tensor([2., 3.], requires_grad=True)
True


In [8]:
# Q의 오른쪽 그래프를 따라갑니다.
print(Q) # 3*a^3 - b^2
print(Q.grad_fn) # -
print(Q.grad_fn.next_functions[1][0]) # ^2
print(Q.grad_fn.next_functions[1][0].next_functions[0][0]) # b 래퍼
print(Q.grad_fn.next_functions[1][0].next_functions[0][0].variable) # b
print(Q.grad_fn.next_functions[1][0].next_functions[0][0].variable is b) # True

tensor([-12.,  65.], grad_fn=<SubBackward0>)
tensor([6., 4.], requires_grad=True)
True


### 연산 그래프에서 배제하기

- 텐서를 생성할 때 `requires_grad=False`로 설정하면 연산 그래프에 기록이 되지 않으며, 따라서 그래디언트를 계산하지도 않는다.
- 이항 연산에 연루된 두 개의 텐서 중 어느 한 쪽이라도 `requires_grad=True`이라면 연산의 결과 얻은 텐서도 `requires_grad=True`로 설정된다.

In [9]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y # x, y 둘 다 gard가 필요없어요
print(f"Does `a` require gradients? : {a.requires_grad}") # 그래서 a도 그라드가 필요없어요
b = x + z # z 는 grad가 필요해요
print(f"Does `b` require gradients?: {b.requires_grad}") # 그래서 b도 grad가 필요해요

Does `a` require gradients? : False
Does `b` require gradients?: True


- 인공신경망의 파라미터중 그래디언트를 계산하지 않게 하는 것을 두고 '파라미터를 얼린다'고 한다.(**frozen parameter**)
- 파라미터를 얼리면 그래디언트를 계산하지 않는 만큼 코드의 실행속도가 빨라진다.
- 또한 파인튜닝을 할 때에도 분류 레이어를 제외한 모든 모델을 얼려두고서 새로운 레이블에 대헤 예측시킨다.
- 예를 들어 다음 코드는 resnet18 모델의 모든 레이어를 얼린 뒤, `model.fc`만 새로운 레이어로 교환해 혼자 훈련이 되도록 만든다.

In [10]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Linear(512, 10)

# Optimize only the classifier
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)